In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install --upgrade pip
!pip install accelerate transformers peft bitsandbytes sentencepiece
!pip install sentence-transformers chromadb faiss-cpu
!pip install pyarrow==19.0.0 datasets==2.14.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 117.6 MB/s  0:00:00m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 87.1 MB/s  0:00:03m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 143.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 54.4 MB/s  0:00:05m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 86.9 MB/s  0:00:02m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 110.5 MB/s  0:00:00m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 90.0 MB/s  0:00:01m0:00:01

In [2]:
import torch
import transformers, peft

print("Torch:", torch.__version__)
print("GPU available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))

print("Transformers:", transformers.__version__)
print("PEFT:", peft.__version__)


2025-11-15 08:41:48.617484: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763196108.769053      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763196108.811268      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

Torch: 2.6.0+cu124
GPU available: True
GPU: Tesla T4
Transformers: 4.53.3
PEFT: 0.16.0


In [3]:
from bitsandbytes import nn as bnbnn
print("bitsandbytes OK")


bitsandbytes OK


In [4]:
import os

dirs = [
    "/kaggle/working/data/fine_tune",
    "/kaggle/working/data/rag_corpus",
    "/kaggle/working/outputs/lora_adapters",
    "/kaggle/working/outputs/checkpoints"
]

for d in dirs:
    os.makedirs(d, exist_ok=True)

dirs


['/kaggle/working/data/fine_tune',
 '/kaggle/working/data/rag_corpus',
 '/kaggle/working/outputs/lora_adapters',
 '/kaggle/working/outputs/checkpoints']

In [5]:
import json

samples = [
    {
        "instruction": "Answer the medical question using safe triage logic and structured explanation.",
        "input": "I have persistent chest tightness for two weeks. What could it be?",
        "output": "### Summary\nChest tightness lasting two weeks can come from several sources.\n\n### Possible Common Causes\n- Stress-related muscle tension\n- Acid reflux (GERD)\n- Chest wall inflammation (costochondritis)\n\n### Possible Serious Causes\n- Heart-related conditions, especially if associated with sweating or radiating pain\n- Lung issues like pulmonary embolism (rare but serious)\n\n### Recommended Next Steps\n- Seek emergency care if symptoms occur with shortness of breath, fainting, or radiating pain.\n- If stable, schedule a medical evaluation for further testing.\n\n### Safety Note\nThis is informational, not a diagnosis."
    },
    {
        "instruction": "Explain the purpose and safety considerations of a medication using clinical tone.",
        "input": "What is ibuprofen used for?",
        "output": "### Purpose\nIbuprofen is a non-steroidal anti-inflammatory drug used for pain, fever, and inflammation.\n\n### Safety Considerations\n- Avoid if you have kidney disease or a history of stomach ulcers.\n- Avoid combining with other NSAIDs.\n- Follow dosage instructions carefully.\n\n### When to Seek Help\nSevere stomach pain, black stools, or vomiting blood require urgent care.\n\n### Safety Note\nAlways follow instructions from a healthcare professional."
    },
    {
        "instruction": "Use the provided medical context to answer safely and concisely.",
        "input": "User question: What does wheezing usually indicate?\n\nContext:\n- Wheezing is commonly associated with asthma, COPD, and allergic reactions.\n- Severe wheezing with shortness of breath is a medical emergency.",
        "output": "### Summary\nWheezing usually indicates an issue with airflow in the lungs.\n\n### Based on Retrieved Medical Context\n- It commonly appears in asthma, COPD, and allergic reactions.\n- Severe wheezing with difficulty breathing requires urgent evaluation.\n\n### Recommended Action\nSeek care promptly if symptoms worsen.\n\n### Safety Note\nThis is for general information only."
    },
    {
        "instruction": "Provide a structured explanation of a symptom and possible causes.",
        "input": "Why do I feel dizzy when I stand up?",
        "output": "### Summary\nFeeling dizzy on standing can occur when blood pressure briefly drops.\n\n### Possible Causes\n- Dehydration\n- Sudden standing after sitting or lying for long periods\n- Low blood pressure (orthostatic hypotension)\n\n### When It Might Be Serious\n- If dizziness leads to fainting\n- If associated with chest pain or irregular heartbeat\n\n### Recommended Next Steps\nIncrease hydration, stand slowly, and seek medical advice if dizziness persists.\n\n### Safety Note\nThis is not a diagnosis."
    },
    {
        "instruction": "Explain a medical condition using a structured clinical style.",
        "input": "What is anemia?",
        "output": "### Summary\nAnemia is a condition where the body has fewer healthy red blood cells than normal.\n\n### Common Causes\n- Iron deficiency\n- Vitamin B12 or folate deficiency\n- Chronic illness\n\n### Possible Symptoms\n- Fatigue\n- Pale skin\n- Shortness of breath on exertion\n\n### Recommended Evaluation\nA blood test is required to confirm the cause.\n\n### Safety Note\nConsult a clinician for proper assessment."
    },
    {
        "instruction": "Use retrieved context to guide the explanation.",
        "input": "User question: Is migraine linked to sensitivity to light?\n\nContext:\n- Light sensitivity (photophobia) is a common symptom associated with migraines.",
        "output": "### Summary\nMigraines frequently involve sensitivity to light.\n\n### Based on Retrieved Context\nPhotophobia is commonly reported during migraine episodes.\n\n### Recommended Action\nIf headaches are frequent or disabling, seek medical evaluation.\n\n### Safety Note\nGeneral informational guidance only."
    },
    {
        "instruction": "Provide a structured response based on medical triage principles.",
        "input": "My heart is beating very fast suddenly. Should I worry?",
        "output": "### Summary\nA sudden rapid heartbeat can have many causes.\n\n### Possible Common Causes\n- Anxiety\n- Caffeine intake\n- Dehydration\n\n### Possible Serious Causes\n- Arrhythmias\n- Thyroid issues\n\n### Recommended Next Steps\nIf rapid heartbeat is accompanied by chest pain, dizziness, or fainting, seek emergency care. Otherwise, monitor and consult a doctor if episodes repeat.\n\n### Safety Note\nThis is general guidance, not a diagnosis."
    },
    {
        "instruction": "Provide a safe explanation using RAG context.",
        "input": "User question: What does swollen lymph nodes usually indicate?\n\nContext:\n- Lymph nodes commonly swell during infections.\n- Persistent swelling requires evaluation.",
        "output": "### Summary\nSwollen lymph nodes typically appear when the immune system is active.\n\n### Based on Retrieved Context\n- Infections are the most common cause.\n- Persistent swelling may require further examination.\n\n### Recommended Action\nMonitor size and tenderness; seek medical advice if swelling lasts more than a few weeks.\n\n### Safety Note\nThis is informational only."
    },
    {
        "instruction": "Explain a symptom with structured, safe reasoning.",
        "input": "Why do I keep getting a sore throat?",
        "output": "### Summary\nRepeated sore throats can result from infection or irritation.\n\n### Possible Causes\n- Viral infections\n- Allergies\n- Exposure to irritants like smoke\n\n### When It Might Be Serious\n- Persistent symptoms\n- Fever lasting more than a few days\n\n### Recommended Next Steps\nHydration, rest, and seek medical care if symptoms do not improve.\n\n### Safety Note\nGeneral information only."
    },
    {
        "instruction": "Use safe clinical reasoning with structured output.",
        "input": "My blood sugar suddenly dropped yesterday. Why would that happen?",
        "output": "### Summary\nA sudden drop in blood sugar can occur from several factors.\n\n### Possible Causes\n- Skipping meals\n- Intense exercise\n- Diabetes medications\n\n### Concerning Signs\n- Confusion\n- Loss of consciousness\n\n### Recommended Next Steps\nMonitor blood sugar frequently and discuss recent patterns with a clinician.\n\n### Safety Note\nThis is not a substitute for medical advice."
    },
    {
        "instruction": "Explain the meaning of a lab test result safely.",
        "input": "What does a high white blood cell count mean?",
        "output": "### Summary\nA high white blood cell count often indicates that the body is responding to something.\n\n### Possible Causes\n- Infection\n- Stress\n- Inflammation\n\n### Recommended Next Steps\nA clinician will evaluate the trend and symptoms to determine the cause.\n\n### Safety Note\nLab results require professional interpretation."
    },
    {
        "instruction": "Generate a structured medical explanation using clinical tone.",
        "input": "What is dehydration, and what are early signs?",
        "output": "### Summary\nDehydration occurs when the body loses more fluids than it takes in.\n\n### Early Signs\n- Dry mouth\n- Thirst\n- Dizziness\n\n### When It Becomes Serious\n- Rapid heartbeat\n- Confusion\n\n### Recommended Next Steps\nIncrease fluid intake and seek medical attention if severe symptoms appear.\n\n### Safety Note\nGeneral guidance only."
    },
    {
        "instruction": "Use context to explain the condition safely.",
        "input": "User question: What does shortness of breath after mild activity mean?\n\nContext:\n- Shortness of breath may be related to lung or heart conditions.\n- Sudden or severe symptoms require urgent care.",
        "output": "### Summary\nShortness of breath after mild activity can indicate reduced tolerance to exertion.\n\n### Based on Retrieved Context\nIt may relate to lung or heart conditions.\n\n### Recommended Action\nSeek medical evaluation, especially if symptoms worsen or appear suddenly.\n\n### Safety Note\nThis is informational only."
    },
    {
        "instruction": "Provide a structured explanation of a common medical question.",
        "input": "Why do I keep feeling tired even after sleeping well?",
        "output": "### Summary\nFatigue despite adequate sleep has multiple possible causes.\n\n### Possible Causes\n- Stress or mental fatigue\n- Poor sleep quality\n- Nutrient deficiencies\n\n### When to Seek Care\nIf symptoms last more than a few weeks or interfere with daily activities.\n\n### Recommended Next Steps\nMonitor patterns and discuss with a clinician.\n\n### Safety Note\nThis is not a diagnosis."
    },
    {
        "instruction": "Explain a symptom using safe medical reasoning.",
        "input": "Why does my heart skip a beat sometimes?",
        "output": "### Summary\nA sensation of skipped heartbeats can come from brief changes in rhythm.\n\n### Common Causes\n- Stress\n- Caffeine\n- Lack of sleep\n\n### When It May Be Serious\n- Frequent episodes\n- Associated dizziness or chest pain\n\n### Recommended Next Steps\nAvoid stimulants and seek medical evaluation if symptoms persist.\n\n### Safety Note\nGeneral information only."
    }
]

# Save file
with open("/kaggle/working/data/fine_tune/medical_instructions.jsonl", "w") as f:
    for item in samples:
        f.write(json.dumps(item) + "\n")

print("Dataset saved to /kaggle/working/data/fine_tune/medical_instructions.jsonl")


Dataset saved to /kaggle/working/data/fine_tune/medical_instructions.jsonl


In [6]:
from datasets import load_dataset

medmcqa = load_dataset("medmcqa", "default")
medmcqa


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/85.9M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/936k [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/1.48M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/182822 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6150 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4183 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
        num_rows: 182822
    })
    test: Dataset({
        features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
        num_rows: 6150
    })
    validation: Dataset({
        features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
        num_rows: 4183
    })
})

In [7]:
medmcqa["train"][0]


{'id': 'e9ad821a-c438-4965-9f77-760819dfa155',
 'question': 'Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma',
 'opa': 'Hyperplasia',
 'opb': 'Hyperophy',
 'opc': 'Atrophy',
 'opd': 'Dyplasia',
 'cop': 2,
 'choice_type': 'single',
 'exp': 'Chronic urethral obstruction because of urinary calculi, prostatic hyperophy, tumors, normal pregnancy, tumors, uterine prolapse or functional disorders cause hydronephrosis which by definition is used to describe dilatation of renal pelvis and calculus associated with progressive atrophy of the kidney due to obstruction to the outflow of urine Refer Robbins 7yh/9,1012,9/e. P950',
 'subject_name': 'Anatomy',
 'topic_name': 'Urinary tract'}

In [8]:
def convert_medmcqa(example):
    # Build question text
    q = example["question"]
    options = f"(A) {example['opa']}\n(B) {example['opb']}\n(C) {example['opc']}\n(D) {example['opd']}"
    
    # Correct answer letter
    correct = example["cop"]
    
    # Explanation provided by dataset
    explanation = example["exp"]
    
    instruction = "Answer the medical multiple-choice question with structured reasoning."
    
    input_text = f"Question: {q}\n\nOptions:\n{options}"
    
    output_text = (
        f"### Summary\nThis is a medical multiple-choice question.\n\n"
        f"### Reasoning\n{explanation}\n\n"
        f"### Correct Answer\nOption {correct}\n\n"
        f"### Safety Note\nThis is educational content, not a diagnosis."
    )
    
    return {
        "instruction": instruction,
        "input": input_text,
        "output": output_text
    }


In [9]:
medmcqa_converted = medmcqa.map(convert_medmcqa)


Map:   0%|          | 0/182822 [00:00<?, ? examples/s]

Map:   0%|          | 0/6150 [00:00<?, ? examples/s]

Map:   0%|          | 0/4183 [00:00<?, ? examples/s]

In [10]:
import json

custom = []
with open("/kaggle/working/data/fine_tune/medical_instructions.jsonl") as f:
    for line in f:
        custom.append(json.loads(line))


In [11]:
from datasets import Dataset

custom_ds = Dataset.from_list(custom)


In [12]:
from datasets import concatenate_datasets

train_ds = concatenate_datasets([medmcqa_converted["train"], custom_ds])
eval_ds = medmcqa_converted["validation"]
test_ds = medmcqa_converted["test"]


In [13]:
train_ds

Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name', 'instruction', 'input', 'output'],
    num_rows: 182837
})

In [14]:
from transformers import AutoTokenizer

model_name = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [15]:
def tokenize(batch):
    full_prompts = []
    for instr, inp, out in zip(batch["instruction"], batch["input"], batch["output"]):
        full_prompts.append(
            f"Instruction: {instr}\n"
            f"Input: {inp}\n"
            f"Output: {out}"
        )
    
    tokens = tokenizer(
        full_prompts,
        truncation=True,
        padding=False,
        max_length=512
    )
    
    return {
        "input_ids": tokens["input_ids"],
        "attention_mask": tokens["attention_mask"]
    }



In [16]:
train_tok = train_ds.map(
    tokenize,
    batched=True,
    remove_columns=train_ds.column_names,
    load_from_cache_file=False
)

eval_tok = eval_ds.map(
    tokenize,
    batched=True,
    remove_columns=eval_ds.column_names,
    load_from_cache_file=False
)


Map:   0%|          | 0/182837 [00:00<?, ? examples/s]

Map:   0%|          | 0/4183 [00:00<?, ? examples/s]

In [17]:
print(train_tok[0].keys())
print(type(train_tok[0]["input_ids"]))
print(len(train_tok[0]["input_ids"]))
print(len(train_tok[0]["attention_mask"]))


dict_keys(['input_ids', 'attention_mask'])
<class 'list'>
242
242


In [18]:
train_tok.save_to_disk("/kaggle/working/data/fine_tune/processed/train")
eval_tok.save_to_disk("/kaggle/working/data/fine_tune/processed/eval")


Saving the dataset (0/1 shards):   0%|          | 0/182837 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4183 [00:00<?, ? examples/s]

In [19]:
medmcqa_converted.save_to_disk("/kaggle/working/data/fine_tune")

Saving the dataset (0/1 shards):   0%|          | 0/182822 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/6150 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4183 [00:00<?, ? examples/s]

In [20]:
import shutil

shutil.make_archive("/kaggle/working/med_rag_data_backup", 'zip', "/kaggle/working/data")


'/kaggle/working/med_rag_data_backup.zip'

In [21]:
import pyarrow
import datasets
print(pyarrow.__version__)
print(datasets.__version__)


19.0.1
4.4.1


In [24]:
import shutil

shutil.copytree(
    "/kaggle/input/med-rag-data-backup/fine_tune",
    "/kaggle/working/data/fine_tune",
    dirs_exist_ok=True
)


'/kaggle/working/data/fine_tune'

In [25]:
import os
os.listdir("/kaggle/working/data/fine_tune")



['test',
 'medical_instructions.jsonl',
 'train',
 'validation',
 'processed',
 'dataset_dict.json']

In [26]:
os.listdir("/kaggle/working/data/fine_tune/processed")


['eval', 'train']

In [27]:
from datasets import load_from_disk

train_tok = load_from_disk("/kaggle/working/data/fine_tune/processed/train")
eval_tok  = load_from_disk("/kaggle/working/data/fine_tune/processed/eval")

print(train_tok[0].keys())
print(len(train_tok))
print(len(eval_tok))


dict_keys(['input_ids', 'attention_mask'])
182837
4183


In [28]:
# Reduce train + eval sizes
train_tok_small = train_tok.shuffle(seed=42).select(range(2000))
eval_tok_small  = eval_tok.shuffle(seed=42).select(range(200))

len(train_tok_small), len(eval_tok_small)

train_tok_small.save_to_disk("/kaggle/working/data/fine_tune/processed/train_small")
eval_tok_small.save_to_disk("/kaggle/working/data/fine_tune/processed/eval_small")


Saving the dataset (0/1 shards):   0%|          | 0/2000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/200 [00:00<?, ? examples/s]

In [29]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
    attn_implementation="sdpa"
)


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [30]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj","k_proj","v_proj","o_proj"],
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 6,815,744 || all params: 7,248,547,840 || trainable%: 0.0940


In [31]:
from transformers import DataCollatorForLanguageModeling

collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)


In [38]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/kaggle/working/outputs/checkpoints",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,   # effective batch = 8, stable on P100
    num_train_epochs=2,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=25,
    save_steps=25,
    eval_strategy="steps",
    eval_steps=25,
    save_total_limit=2,
    dataloader_num_workers=2,
    report_to="none"
)


In [39]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tok_small,
    eval_dataset=eval_tok_small,
    data_collator=collator
)


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [40]:
trainer.train()
model.save_pretrained("/kaggle/working/outputs/lora_adapters/")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Step,Training Loss,Validation Loss
25,1.323200,1.060156
50,1.262500,0.997880
75,1.225900,0.975653
100,1.239700,0.968084
125,1.228800,0.966978
150,1.151700,0.968964
175,1.092700,0.968871
200,1.099000,0.970759
225,1.100200,0.966853
250,1.090800,0.967216


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [42]:
# 1. Save LoRA adapter
model.save_pretrained("/kaggle/working/best_lora_model")

# 2. Save tokenizer (optional but recommended)
tokenizer.save_pretrained("/kaggle/working/best_lora_model")


('/kaggle/working/best_lora_model/tokenizer_config.json',
 '/kaggle/working/best_lora_model/special_tokens_map.json',
 '/kaggle/working/best_lora_model/chat_template.jinja',
 '/kaggle/working/best_lora_model/tokenizer.model',
 '/kaggle/working/best_lora_model/added_tokens.json',
 '/kaggle/working/best_lora_model/tokenizer.json')

In [48]:
import os

os.listdir("/kaggle/working/best_lora_model")


['tokenizer_config.json',
 'chat_template.jinja',
 'adapter_model.safetensors',
 'adapter_config.json',
 'README.md',
 'special_tokens_map.json',
 'tokenizer.json',
 'tokenizer.model']

In [49]:
import os

os.listdir("/kaggle/working/outputs/lora_adapters/")


['adapter_model.safetensors', 'adapter_config.json', 'README.md']

In [50]:
import shutil
shutil.make_archive("/kaggle/working/best_lora_model", 'zip', "/kaggle/working/best_lora_model")


'/kaggle/working/best_lora_model.zip'